In [2]:
#Code by David Roberts https://www.kaggle.com/code/davidbroberts/tensorflow-transfer-learning/notebook

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Set the log level to keep the warnings down
import torch
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from lib.datasets.datasets import BinMaze
from tqdm import tqdm
from lib.datasets.datasets import get_maze_data
from torch.utils.data import Dataset, DataLoader
import torchvision
from config.config_hollow_maze_distr import get_config 
from lib.datasets.maze import maze_gen
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt
import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.datasets as datasets
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.loggers.loggers as loggers
import lib.loggers.logger_utils as logger_utils

In [3]:
num_gpus = torch.cuda.device_count()
print("Anzahl der verfügbaren GPUs:", num_gpus)
def get_script_dir():
    return os.path.dirname(os.path.realpath(__file__))
print(get_script_dir())

Anzahl der verfügbaren GPUs: 0


NameError: name '__file__' is not defined

In [26]:
save_location = '/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/TAUnSDDM/SavedModels/'
experiment_name = 'MAZE'
cfg = get_config()
save_dir, checkpoint_dir, config_dir = \
                bookkeeping.create_experiment_folder(
                    cfg.save_location,
                    cfg.experiment_name,
                    False
            )
bookkeeping.save_config_as_yaml(cfg, config_dir)
bookkeeping.save_git_hash(save_dir)
print(save_dir)

/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/TAUnSDDM/SavedModels/2023-11-01/MAZE


In [20]:
model = model_utils.create_model(cfg, device='cpu')
print("number of parameters: ", sum([p.numel() for p in model.parameters()]))
optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)
state = {
        'model': model,
        'optimizer': optimizer,
        'n_iter': 0
    }
bookkeeping.setup_preemption(
            save_dir, checkpoint_dir, state,
            2,
            True
        )

number of parameters:  439490


In [21]:
preempted_path = Path("null")
preemption_start_day_YYYYhyphenMMhyphenDD = '2023-10-31'
preempted_path = bookkeeping.check_for_preempted_run(
    cfg.save_location,
    preemption_start_day_YYYYhyphenMMhyphenDD,
    cfg,
    True
)
print(preempted_path.as_posix())

Bookkeeping: checking for preempted run
Bookkeeping: no preempted run found
null


In [22]:
# Eine Funktion, um einige Bilder zu plotten
def show_images(images, n=8):
    plt.figure(figsize=(15, 15))
    for i in range(n):
        ax = plt.subplot(1, n, i + 1)
        plt.imshow(images[i].permute(1, 2, 0).numpy().astype("uint8"), cmap='gray')
        plt.axis("off")
    plt.show()
for i in range(3 -1):
    print(i)
    if i != (3-2):
        print("act")

0
act
1


In [23]:
config = get_config()
img = maze_gen(limit=10, dim_x=7, dim_y=7, pixelSizeOfTile=2, weightHigh=97,weightLow=97)
torch_dataloader = get_maze_data(config, img)

AttributeError: 'list' object has no attribute 'to'

In [ ]:
for i in tqdm(torch_dataloader):
    print(i, i.shape)
    break

In [ ]:


# Nehmen Sie einen Batch von Bildern
images = next(iter(torch_dataloader))
show_images(images * 255, n=10)